In [ ]:
import React, { useState, useEffect, useRef } from 'react';
import { Book, Wand2, Battery, Send, RefreshCw, ShoppingBag, Shield, Heart, Zap, Brain, Lightbulb, Target, Smile, Flame, Droplets, Leaf, Gift, Skull, Wind, Moon, Info, X, Dumbbell, Coffee, Briefcase, Sun, Thermometer } from 'lucide-react';

// ==========================================
// Google Gemini API 설정
// ==========================================
const apiKey = ""; // 런타임 환경에서 자동으로 주입됩니다.

// ==========================================
// 데이터베이스: 몬스터 & 실생활 시나리오
// ==========================================
const class2Monsters = [
  { 
    id: 'm1', name: "Fire Spirit", type: "FIRE", weakness: ["cold", "cool", "icy", "wet"], 
    hp: 150, look: <Flame size={80} className="text-orange-500 animate-bounce"/>,
    desc: "불타는 정령입니다. 차가운(Cold) 공격이 필요합니다.", category: 'MONSTER'
  },
  { 
    id: 'm2', name: "Water Slime", type: "WATER", weakness: ["hot", "warm", "dry", "electric"], 
    hp: 120, look: <Droplets size={80} className="text-blue-500 animate-pulse"/>,
    desc: "축축한 슬라임입니다. 뜨거운(Hot) 공격에 약합니다.", category: 'MONSTER'
  },
  { 
    id: 'm3', name: "Iron Golem", type: "METAL", weakness: ["strong", "heavy", "hard", "hot"], 
    hp: 200, look: <Dumbbell size={80} className="text-gray-400 animate-bounce"/>,
    desc: "단단한 강철 골렘입니다. 강한(Strong) 충격이나 녹이는 열이 필요합니다.", category: 'MONSTER'
  },
  { 
    id: 'm4', name: "Wind Bat", type: "WIND", weakness: ["heavy", "fast", "quick"], 
    hp: 100, look: <Wind size={80} className="text-sky-300 animate-pulse"/>,
    desc: "재빠른 박쥐입니다. 무거운(Heavy) 바람으로 누르거나 더 빨라야(Fast) 합니다.", category: 'MONSTER'
  },
  { 
    id: 'm5', name: "Dark Shadow", type: "DARK", weakness: ["bright", "shiny", "light"], 
    hp: 130, look: <Moon size={80} className="text-purple-900 animate-pulse"/>,
    desc: "어둠의 그림자입니다. 밝은(Bright) 빛이 약점입니다.", category: 'MONSTER'
  }
];

// 실생활 적용 시나리오 (3스테이지마다 등장)
const realLifeScenarios = [
  {
    id: 'r1', name: "Too Hot Coffee", type: "TOO HOT", weakness: ["cold", "cool", "iced"],
    hp: 80, look: <Coffee size={80} className="text-amber-700 animate-bounce"/>,
    desc: "커피가 너무 뜨거워서 마실 수 없습니다! 식혀주세요.", category: 'REAL_LIFE'
  },
  {
    id: 'r2', name: "Heavy Luggage", type: "HEAVY", weakness: ["light", "strong"], // Light to make it light, Strong to lift it
    hp: 100, look: <Briefcase size={80} className="text-brown-600"/>,
    desc: "짐이 너무 무거워서 들 수 없습니다. 가볍게 만들거나(Light) 힘을 쓰세요(Strong).", category: 'REAL_LIFE'
  },
  {
    id: 'r3', name: "Dark Room", type: "DARKNESS", weakness: ["bright", "light"],
    hp: 60, look: <Sun size={80} className="text-gray-900 bg-yellow-100 rounded-full p-2"/>, // Represents light needed in dark
    desc: "방이 너무 어둡습니다. 불을 켜거나 밝게 만들어주세요.", category: 'REAL_LIFE'
  }
];

// 유물 (패시브 효과)
const artifacts = [
  { id: 'Potion', name: 'Health Potion', icon: <Heart className="text-red-500"/>, desc: "즉시 HP 50 회복 (1회용)", type: "CONSUMABLE" },
  { id: 'ManaCrystal', name: 'Mana Crystal', icon: <Battery className="text-yellow-500"/>, desc: "최대 마나 +20 (지속)", type: "PASSIVE" },
  { id: 'GrimoirePage', name: 'Secret Page', icon: <Book className="text-purple-500"/>, desc: "몬스터 약점 자동 분석 (지속)", type: "PASSIVE" }
];

// 퀴즈 데이터 (마나 훈련소용)
const vocabQuizzes = [
  { q: "차가운", a: "cold" }, { q: "뜨거운", a: "hot" }, 
  { q: "빠른", a: "fast" }, { q: "무거운", a: "heavy" },
  { q: "밝은", a: "bright" }, { q: "강한", a: "strong" },
  { q: "날카로운", a: "sharp" }, { q: "젖은", a: "wet" }
];

// ==========================================
// AI Logic
// ==========================================
async function getWizFeedback(userText, currentMonster) {
  try {
    const isRealLife = currentMonster.category === 'REAL_LIFE';
    const systemPrompt = `
      당신은 AI 튜터 '위즈'입니다. 로그라이크 게임의 심판입니다.

      [현재 상황: ${isRealLife ? "실생활 문제 해결" : "몬스터 전투"}]
      - 대상: ${currentMonster.name} (속성/상태: ${currentMonster.type})
      - 유효 해결 단어(반의어 등): ${currentMonster.weakness.join(', ')}
      - 목표 구문: S + V + Adjective + O (예: I make it cool, I cast cold ice)
      
      [판정 기준]
      1. 문법(어순)이 대략적으로 맞아야 함.
      2. **형용사(Adjective)**가 대상의 상태를 해결하거나 반대되는 개념(반의어)이면 "Critical Hit".
      3. 형용사가 없거나 관련 없으면 "Normal Hit" (데미지 낮음).
      4. 문맥상 완전히 틀리면 "Miss".
      
      [Output JSON]
      {
        "isCorrect": boolean,
        "damage": number (0=Miss, 20=Normal, 60=Critical),
        "message": "위즈의 피드백 (한국어, 이모지 포함, ${isRealLife ? "실생활 조언 톤으로" : "전투 톤으로"})",
        "scaffoldingType": "Conceptual" | "Strategic" | "Motivational" | "Success"
      }
    `;

    const response = await fetch(
      `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent?key=${apiKey}`,
      {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({
          contents: [{ parts: [{ text: `Check this input against target '${currentMonster.name}': ${userText}` }] }],
          systemInstruction: { parts: [{ text: systemPrompt }] },
          generationConfig: { responseMimeType: "application/json" }
        }),
      }
    );

    if (!response.ok) throw new Error("API Error");
    const data = await response.json();
    return JSON.parse(data.candidates[0].content.parts[0].text);

  } catch (error) {
    console.error(error);
    return { isCorrect: false, message: "마법 회로 불안정! 다시 시도해.", damage: 0 };
  }
}

// ==========================================
// Main Component
// ==========================================
export default function Class2_Roguelike() {
  const [phase, setPhase] = useState('INTRO'); // INTRO, COMBAT, REWARD, GAME_OVER
  
  // Player Stats
  const [playerHp, setPlayerHp] = useState(100);
  const [maxHp, setMaxHp] = useState(100);
  const [mana, setMana] = useState(50);
  const [maxMana, setMaxMana] = useState(50);
  const [inventory, setInventory] = useState([]); // List of artifacts
  
  // Session State
  const [messages, setMessages] = useState([]);
  const [userInput, setUserInput] = useState('');
  const [loading, setLoading] = useState(false);
  const [currentMonster, setCurrentMonster] = useState(null);
  const [stageCount, setStageCount] = useState(1);
  const [monsterHp, setMonsterHp] = useState(100);

  // UI Toggles
  const [showGrimoire, setShowGrimoire] = useState(false);
  const [showManaStation, setShowManaStation] = useState(false);

  const messagesEndRef = useRef(null);

  // Initialize
  useEffect(() => {
    // 메시지 스크롤
    messagesEndRef.current?.scrollIntoView({ behavior: "smooth" });
  }, [messages]);

  const startStage = () => {
    let targetEntity;
    let introMsg;

    // 3스테이지마다 실생활 미션 등장
    if (stageCount % 3 === 0) {
        targetEntity = realLifeScenarios[Math.floor(Math.random() * realLifeScenarios.length)];
        introMsg = `🏙️ 실생활 미션 발생! ${targetEntity.name} 상황을 해결하세요!`;
    } else {
        targetEntity = class2Monsters[Math.floor(Math.random() * class2Monsters.length)];
        introMsg = `⚠️ 야생의 ${targetEntity.name}(이)가 나타났다!`;
    }

    setCurrentMonster({ ...targetEntity, maxHp: targetEntity.hp }); 
    setMonsterHp(targetEntity.hp);
    setPhase('COMBAT');
    setMessages(prev => [...prev, { 
      sender: 'system', 
      text: introMsg 
    }]);
  };

  const startGame = () => {
    setPhase('COMBAT');
    setMessages([{ sender: 'wiz', text: "좋아, 숲으로 들어가자! 상황에 맞는 '반의어 형용사'를 사용해서 문제를 해결해야 해." }]);
    startStage();
  }

  const handleSendMessage = async () => {
    if (!userInput.trim()) return;

    if (mana < 5) {
        setMessages(prev => [...prev, { sender: 'system', text: "⚠️ 마나가 부족합니다! '마나 훈련소'에서 충전하세요." }]);
        return;
    }

    const newMessages = [...messages, { sender: 'user', text: userInput }];
    setMessages(newMessages);
    setLoading(true);
    setUserInput('');
    setMana(prev => Math.max(0, prev - 5)); // 마나 소모

    const result = await getWizFeedback(userInput, currentMonster);
    setLoading(false);
    
    setMessages(prev => [...prev, { sender: 'wiz', text: result.message, scaffoldingType: result.scaffoldingType }]);

    if (result.isCorrect) {
      const dmg = result.damage || 20;
      const newHp = Math.max(0, monsterHp - dmg);
      setMonsterHp(newHp);

      if (dmg >= 60) {
        setMessages(prev => [...prev, { sender: 'system', text: "🔥 Critical Hit! 완벽한 해결책입니다!" }]);
      }

      if (newHp <= 0) {
        setStageCount(prev => prev + 1);
        setPhase('REWARD');
        const clearMsg = currentMonster.category === 'REAL_LIFE' 
            ? `✅ 문제 해결 완료! ${currentMonster.name} 상황을 극복했습니다.`
            : `🏆 ${currentMonster.name} 처치! 보상을 선택하세요.`;
        setMessages(prev => [...prev, { sender: 'system', text: clearMsg }]);
      } else {
        // Monster/Scenario Counter Attack/Effect
        const damageAmount = Math.floor(Math.random() * 10) + 5;
        setPlayerHp(prev => Math.max(0, prev - damageAmount));
        
        const attackMsg = currentMonster.category === 'REAL_LIFE'
            ? `💦 상황이 악화되었습니다. 스트레스를 받습니다. (HP -${damageAmount})`
            : `💥 몬스터가 반격합니다! (HP -${damageAmount})`;

        setMessages(prev => [...prev, { sender: 'system', text: attackMsg }]);
        if (playerHp - damageAmount <= 0) setPhase('GAME_OVER');
      }
    } else {
       const damageAmount = 15;
       setPlayerHp(prev => Math.max(0, prev - damageAmount));
       const failMsg = currentMonster.category === 'REAL_LIFE'
        ? `❌ 해결 실패! 상황이 더 꼬였습니다. (HP -${damageAmount})`
        : `❌ 주문 실패! 몬스터에게 강하게 맞았습니다. (HP -${damageAmount})`;

       setMessages(prev => [...prev, { sender: 'system', text: failMsg }]);
       if (playerHp - damageAmount <= 0) setPhase('GAME_OVER');
    }
  };

  const selectArtifact = (artifact) => {
    if (artifact.type === 'CONSUMABLE') {
        if (artifact.id === 'Potion') setPlayerHp(prev => Math.min(maxHp, prev + 50));
    } else {
        // Passive items
        setInventory(prev => [...prev, artifact]);
        if (artifact.id === 'ManaCrystal') {
            setMaxMana(prev => prev + 20);
            setMana(prev => prev + 20); // Fill the extra mana
        }
    }
    
    setMessages(prev => [...prev, { sender: 'system', text: `🎁 ${artifact.name} 획득! 다음 스테이지로 이동합니다...` }]);
    startStage();
  };

  // Sub-Components
  const GrimoireModal = () => (
    <div className="fixed inset-0 bg-black/80 z-50 flex items-center justify-center p-4">
      <div className="bg-slate-800 text-slate-100 rounded-lg max-w-2xl w-full p-6 border border-slate-600 shadow-2xl relative">
        <button onClick={() => setShowGrimoire(false)} className="absolute top-4 right-4"><X /></button>
        <h2 className="text-2xl font-bold mb-4 flex items-center gap-2 text-yellow-400"><Book /> 속성 반의어 가이드</h2>
        
        <div className="grid grid-cols-1 md:grid-cols-2 gap-4">
            <div className="bg-slate-700 p-4 rounded-lg">
                <h3 className="font-bold mb-2 text-orange-400">🔥 불 (Hot/Fire)</h3>
                <p className="text-sm mb-2 text-slate-300">반대 속성 (해결책):</p>
                <div className="flex gap-2 flex-wrap">
                    <span className="bg-blue-900/50 text-blue-200 px-2 py-1 rounded text-xs border border-blue-500">Cold</span>
                    <span className="bg-blue-900/50 text-blue-200 px-2 py-1 rounded text-xs border border-blue-500">Cool</span>
                </div>
            </div>
            <div className="bg-slate-700 p-4 rounded-lg">
                <h3 className="font-bold mb-2 text-blue-400">💧 물/추위 (Cold/Water)</h3>
                <p className="text-sm mb-2 text-slate-300">반대 속성 (해결책):</p>
                <div className="flex gap-2 flex-wrap">
                    <span className="bg-red-900/50 text-red-200 px-2 py-1 rounded text-xs border border-red-500">Hot</span>
                    <span className="bg-red-900/50 text-red-200 px-2 py-1 rounded text-xs border border-red-500">Warm</span>
                    <span className="bg-red-900/50 text-red-200 px-2 py-1 rounded text-xs border border-red-500">Dry</span>
                </div>
            </div>
            <div className="bg-slate-700 p-4 rounded-lg">
                <h3 className="font-bold mb-2 text-gray-400">⚙️ 강철/무거움 (Heavy)</h3>
                <p className="text-sm mb-2 text-slate-300">반대 속성 (해결책):</p>
                <div className="flex gap-2 flex-wrap">
                    <span className="bg-emerald-900/50 text-emerald-200 px-2 py-1 rounded text-xs border border-emerald-500">Light (가벼운)</span>
                    <span className="bg-purple-900/50 text-purple-200 px-2 py-1 rounded text-xs border border-purple-500">Strong (강한)</span>
                </div>
            </div>
            <div className="bg-slate-700 p-4 rounded-lg">
                <h3 className="font-bold mb-2 text-purple-400">🌑 어둠 (Dark)</h3>
                <p className="text-sm mb-2 text-slate-300">반대 속성 (해결책):</p>
                <div className="flex gap-2 flex-wrap">
                    <span className="bg-yellow-900/50 text-yellow-200 px-2 py-1 rounded text-xs border border-yellow-500">Bright</span>
                    <span className="bg-yellow-900/50 text-yellow-200 px-2 py-1 rounded text-xs border border-yellow-500">Shiny</span>
                </div>
            </div>
        </div>
        <div className="mt-4 p-3 bg-yellow-900/30 border border-yellow-600 rounded text-sm text-yellow-200">
            💡 Tip: "Make it <strong>[Adjective]</strong>" 또는 "I use <strong>[Adjective]</strong> magic" 형태로 문장을 만들어보세요.
        </div>
      </div>
    </div>
  );

  const ManaStationModal = () => {
    const [quizIdx, setQuizIdx] = useState(0);
    const [feedback, setFeedback] = useState(null);

    const checkAns = (ans) => {
        if(ans === vocabQuizzes[quizIdx].a) {
            setMana(prev => Math.min(maxMana, prev + 10));
            setFeedback("Correct! (+10 Mana)");
            setTimeout(() => {
                setFeedback(null);
                setQuizIdx((prev) => (prev + 1) % vocabQuizzes.length);
            }, 1000);
        } else {
            setFeedback("Wrong! Try again.");
        }
    };

    return (
      <div className="fixed inset-0 bg-black/80 z-50 flex items-center justify-center p-4">
        <div className="bg-indigo-900 text-white rounded-lg max-w-sm w-full p-6 border-4 border-indigo-500 relative">
          <button onClick={() => setShowManaStation(false)} className="absolute top-2 right-2"><X/></button>
          <h2 className="text-xl font-bold mb-4 flex items-center gap-2"><Battery className="text-yellow-400"/> 마나 훈련소</h2>
          <p className="text-indigo-200 text-sm mb-4">형용사를 맞춰 마나를 충전하세요!</p>
          
          <div className="text-center py-6 bg-indigo-800 rounded-lg mb-4">
            <div className="text-2xl font-bold mb-2">{vocabQuizzes[quizIdx].q}</div>
            <div className="text-sm text-gray-400">영어로 무엇일까요?</div>
            {feedback && <div className="mt-2 text-yellow-300 font-bold animate-pulse">{feedback}</div>}
          </div>

          <div className="grid grid-cols-2 gap-2">
            {[vocabQuizzes[quizIdx].a, "slow", "weak", "dark"].sort(()=>Math.random()-0.5).map((w, i) => (
                <button key={i} onClick={() => checkAns(w)} className="bg-indigo-600 hover:bg-indigo-500 p-3 rounded font-bold">{w}</button>
            ))}
          </div>
        </div>
      </div>
    );
  };

  // Rendering
  if (phase === 'INTRO') {
    return (
      <div className="h-screen bg-slate-900 text-white flex flex-col items-center justify-center p-6 bg-[url('https://images.unsplash.com/photo-1518709268805-4e9042af9f23?q=80&w=2568&auto=format&fit=crop')] bg-cover">
        <div className="absolute inset-0 bg-black/70"></div>
        <div className="z-10 max-w-2xl w-full bg-slate-800/90 p-8 rounded-2xl border-2 border-purple-500 shadow-2xl">
            <h1 className="text-4xl font-bold text-purple-400 mb-6 flex items-center gap-3">
                <Wand2 size={40}/> Class 2: The Forbidden Forest
            </h1>
            
            <div className="space-y-6 text-lg">
                <section>
                    <h2 className="font-bold text-yellow-400 mb-2 flex items-center gap-2"><Target/> 학습 목표</h2>
                    <div className="bg-black/40 p-4 rounded-lg border border-yellow-500/30">
                        <ul className="list-disc pl-5 space-y-2 text-yellow-100">
                            <li><strong>반의어(Antonyms) 학습:</strong> Hot ↔ Cold, Heavy ↔ Light 등의 반대 개념을 익힙니다.</li>
                            <li><strong>상황 해결 능력:</strong> 몬스터 전투 뿐만 아니라, <strong>실생활 문제</strong> 상황에 맞는 형용사를 사용해 봅니다.</li>
                        </ul>
                    </div>
                </section>

                <section>
                    <h2 className="font-bold text-blue-400 mb-2 flex items-center gap-2"><Brain/> 게임 규칙</h2>
                    <ul className="list-disc pl-6 space-y-2 text-slate-300">
                        <li>몬스터의 <strong>속성</strong>을 보고 <strong>반대되는 형용사</strong>로 공격하세요.</li>
                        <li>(예: 불 몬스터 → Cold Magic)</li>
                        <li>3스테이지마다 <strong>실생활 미션</strong>이 등장합니다!</li>
                    </ul>
                </section>

                <button onClick={startGame} className="w-full bg-purple-600 hover:bg-purple-500 text-white font-bold py-4 rounded-xl text-xl transition-transform hover:scale-105 shadow-lg">
                    학습 시작하기
                </button>
            </div>
        </div>
      </div>
    );
  }

  if (phase === 'GAME_OVER') {
    return (
      <div className="h-screen bg-black flex flex-col items-center justify-center text-red-600">
        <Skull size={100} className="mb-4 animate-pulse"/>
        <h1 className="text-4xl font-bold mb-2">GAME OVER</h1>
        <p className="text-gray-400 mb-6">체력이 모두 소진되었습니다.</p>
        <button onClick={() => window.location.reload()} className="mt-8 bg-red-900 px-6 py-3 rounded text-white hover:bg-red-700">다시 도전</button>
      </div>
    );
  }

  return (
    <div className="flex flex-col h-screen bg-slate-900 text-slate-100 font-sans overflow-hidden">
      {/* Header */}
      <header className="bg-slate-800 p-3 shadow-lg flex justify-between items-center border-b border-slate-700 z-20">
        <div className="flex items-center gap-2">
          <div className="bg-purple-600 p-2 rounded-full"><Wand2 size={18} /></div>
          <div>
            <h1 className="text-base font-bold text-purple-200 hidden md:block">Wiz Academy: Roguelike</h1>
            <div className="text-xs text-slate-400">Stage {stageCount}</div>
          </div>
        </div>
        
        {/* Inventory Display */}
        <div className="flex items-center gap-2">
            {inventory.map((item, idx) => (
                <div key={idx} className="bg-slate-700 p-1.5 rounded border border-slate-500" title={item.name}>
                    {item.icon}
                </div>
            ))}
        </div>

        {/* Status Bars */}
        <div className="flex gap-3">
             {/* HP Bar */}
             <div className="flex items-center gap-1 bg-red-900/50 px-3 py-1 rounded-full border border-red-500 text-sm">
                <Heart className="w-4 h-4 text-red-400 fill-red-400" /> 
                <span className="text-red-100 font-bold w-12 text-center">{playerHp}/{maxHp}</span>
             </div>

             {/* Mana Bar */}
             <button onClick={() => setShowManaStation(true)} className="flex items-center gap-1 bg-indigo-900/50 px-3 py-1 rounded-full border border-indigo-500 text-sm hover:bg-indigo-800 transition">
                <Battery className="w-4 h-4 text-yellow-400" /> 
                <span className="text-yellow-200 font-bold w-12 text-center">{mana}/{maxMana}</span>
             </button>
             
             <button onClick={() => setShowGrimoire(true)} className="flex items-center gap-1 bg-amber-900/50 px-3 py-1 rounded-full border border-amber-600 text-sm hover:bg-amber-800 transition">
                <Book className="w-4 h-4 text-amber-400" /> <span className="text-amber-200 hidden sm:inline">Guide</span>
             </button>
        </div>
      </header>

      {/* Main Stage */}
      <div className={`flex-1 relative transition-all duration-1000 flex flex-col items-center justify-center p-4 bg-cover bg-center ${currentMonster?.category === 'REAL_LIFE' ? "bg-[url('https://images.unsplash.com/photo-1493663284031-b7e3aefcae8e?q=80&w=2670&auto=format&fit=crop')]" : "bg-[url('https://images.unsplash.com/photo-1511497584788-876760111969?q=80&w=2670&auto=format&fit=crop')]"}`}>
        <div className="absolute inset-0 bg-slate-900/60 backdrop-blur-[1px]"></div>

        {/* Entity Area (Monster or Situation) */}
        <div className="z-10 mb-6 flex flex-col items-center h-48 justify-end">
            {currentMonster && (
                <div className="flex flex-col items-center animate-fade-in-up">
                    <div className="mb-4 filter drop-shadow-[0_0_15px_rgba(255,255,255,0.3)] transform transition-transform hover:scale-110 duration-300">
                        {currentMonster.look}
                    </div>
                    
                    {/* Weakness Hint (Only if artifact exists) */}
                    {inventory.some(i => i.id === 'GrimoirePage') && (
                        <div className="mb-2 text-xs bg-purple-900/90 px-3 py-1 rounded-full text-purple-200 animate-pulse border border-purple-500 shadow-lg">
                            👁️ Key Word: {currentMonster.weakness[0]}
                        </div>
                    )}

                    <div className="w-48 bg-slate-700 h-4 rounded-full overflow-hidden border border-white/20 shadow-inner">
                        <div className={`h-full transition-all duration-500 ease-out ${currentMonster.category === 'REAL_LIFE' ? 'bg-blue-500' : 'bg-red-500'}`} style={{width: `${(monsterHp/currentMonster.maxHp)*100}%`}}></div>
                    </div>
                    <div className="mt-2 bg-black/70 px-4 py-1.5 rounded-lg text-sm font-bold border border-slate-500 text-white shadow-lg backdrop-blur-sm">
                        {currentMonster.category === 'REAL_LIFE' ? '🏙️ Mission: ' : 'Lv.' + stageCount + ' '}{currentMonster.name}
                    </div>
                </div>
            )}
        </div>

        {/* Chat Area */}
        <div className="z-10 w-full max-w-lg h-60 overflow-y-auto space-y-3 px-2 custom-scrollbar mask-image-gradient">
          {messages.map((msg, idx) => (
            <div key={idx} className={`flex ${msg.sender === 'user' ? 'justify-end' : 'justify-start'}`}>
              <div className={`max-w-[85%] rounded-xl p-3 text-sm shadow-lg ${
                  msg.sender === 'user' ? 'bg-purple-600 text-white rounded-br-none' 
                  : msg.sender === 'system' ? 'bg-black/60 text-yellow-300 text-center w-full border border-yellow-500/30' 
                  : 'bg-slate-800/90 text-slate-200 border border-slate-600 rounded-bl-none'
                }`}>
                {msg.sender === 'wiz' && <div className="flex items-center gap-1 text-xs text-purple-300 font-bold mb-1"><Wand2 size={10}/> Wiz</div>}
                {msg.text}
                {msg.scaffoldingType && <div className="text-[10px] bg-white/10 mt-1 px-1 rounded inline-block">{msg.scaffoldingType} Support</div>}
              </div>
            </div>
          ))}
          <div ref={messagesEndRef} />
        </div>

        {phase === 'REWARD' && (
            <div className="absolute inset-0 bg-black/85 z-50 flex flex-col items-center justify-center p-4">
            <h2 className="text-3xl font-bold text-yellow-400 mb-2 animate-bounce">Stage Cleared!</h2>
            <p className="text-slate-300 mb-8">멋진 해결 능력이군요! 보상을 선택하세요.</p>
            <div className="flex gap-4 flex-wrap justify-center">
                {artifacts.map((art) => (
                <button key={art.id} onClick={() => selectArtifact(art)} className="bg-slate-800 border-2 border-yellow-600 p-6 rounded-xl hover:bg-slate-700 hover:scale-105 transition flex flex-col items-center w-40 shadow-xl group">
                    <div className="mb-4 transform group-hover:scale-125 transition-transform duration-300">{art.icon}</div>
                    <div className="font-bold text-yellow-200">{art.name}</div>
                    <div className="text-xs text-gray-400 mt-2 text-center text-wrap">{art.desc}</div>
                </button>
                ))}
            </div>
            </div>
        )}
      </div>

      {/* Input Area */}
      <div className="bg-slate-800 p-4 border-t border-slate-700 z-30">
        <div className="max-w-xl mx-auto">
            {phase === 'COMBAT' && currentMonster && (
                <div className={`mb-3 p-3 rounded-lg border-l-4 ${currentMonster.category === 'REAL_LIFE' ? 'bg-green-900/20 border-green-500' : 'bg-blue-900/20 border-blue-500'}`}>
                    <div className={`flex justify-between text-xs font-bold uppercase mb-1 ${currentMonster.category === 'REAL_LIFE' ? 'text-green-300' : 'text-blue-300'}`}>
                        <span>{currentMonster.category === 'REAL_LIFE' ? 'Real Life Mission' : 'Survival Mode'}</span>
                        <span>Target: {currentMonster.type}</span>
                    </div>
                    <div className="text-sm text-slate-200">
                        {currentMonster.desc}
                    </div>
                </div>
            )}
            <div className="flex gap-2">
                <input
                    type="text"
                    value={userInput}
                    onChange={(e) => setUserInput(e.target.value)}
                    onKeyPress={(e) => e.key === 'Enter' && handleSendMessage()}
                    placeholder={phase === 'GAME_OVER' ? "게임 오버" : "해결책을 입력하세요 (예: It is too hot)"}
                    disabled={loading || phase === 'REWARD' || phase === 'GAME_OVER'}
                    className="flex-1 bg-slate-900 border border-slate-600 text-white px-4 py-3 rounded-lg focus:ring-2 focus:ring-purple-500 outline-none placeholder-slate-500 transition-all"
                />
                <button onClick={handleSendMessage} disabled={loading || !userInput.trim()} className="bg-purple-600 hover:bg-purple-500 disabled:bg-slate-700 text-white p-3 rounded-lg transition-colors flex items-center justify-center min-w-[3rem]">
                    {loading ? <RefreshCw className="animate-spin w-5 h-5"/> : <Send className="w-5 h-5" />}
                </button>
            </div>
        </div>
      </div>
      
      {showGrimoire && <GrimoireModal />}
      {showManaStation && <ManaStationModal />}
    </div>
  );
}

SyntaxError: invalid syntax (1368465611.py, line 1)